<a href="https://colab.research.google.com/github/Sohaila24/GenAI-Basic-Learnings/blob/main/LlamaIndex_API_using_Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
%pip install llama-index-llms-groq

In [19]:
!pip install llama-index

In [25]:
pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 852.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [27]:
import os
from dotenv import load_dotenv
from llama_index.core import GPTVectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding # use Groq instead of openai
from llama_index.core import ServiceContext

In [28]:
# Load environment variables
load_dotenv()

False

In [29]:
# Set API key for Groq
GROQ_API_KEY = os.environ.get("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")

In [30]:
def build_storage(data_dir, persist_dir):
    """Builds and saves an index from documents."""
    documents = SimpleDirectoryReader(data_dir).load_data()
    index = GPTVectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir)
    return index

In [31]:
def read_from_storage(persist_dir):
    """Loads an existing index from storage."""
    storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
    return load_index_from_storage(storage_context)

In [32]:
def get_storage_index():
    """Retrieves or creates the index."""
    persist_dir = os.environ.get("STORAGE_PATH")
    data_dir = os.environ.get("DATA_PATH")

    if os.path.exists(persist_dir):
        return read_from_storage(persist_dir)
    return build_storage(data_dir, persist_dir)

In [63]:
from llama_index.core.settings import Settings

def get_query_engine():
    """Creates a query engine using Groq LLM and Hugging Face embeddings."""

    # Set global LLM model
    Settings.llm = Groq(model="mixtral-8x7b-32768", temperature=0.2, max_tokens=2000, api_key=os.getenv("GROQ_API_KEY"))

    # Set global embedding model
    Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Define chunk size globally
    Settings.chunk_size = 512

    # Load or build the index
    index = get_storage_index()

    # Create query engine
    query_engine = index.as_query_engine()
    return query_engine

In [34]:
def query_llm(q):
    """Processes a query and returns a response."""
    query_engine = get_query_engine()
    response = query_engine.query(q)
    print(response)
    return response

In [36]:
!pip install Flask

In [71]:
from flask import Flask, request, jsonify, abort
app = Flask(__name__)


In [72]:

# API Endpoint
@app.route(f'{PREFIX}/query', methods=['POST'])
def query():
    """Handles incoming API requests for querying the system."""
    content = request.get_json(silent=True)
    q = content['query']

    try:
        res = query_llm(q)
        return json.dumps({"response": str(res)})
    except Exception as err:
        print(err)
        abort(404)

In [43]:
from google.colab import files
uploaded = files.upload()


Saving Sohaila Mohamed.pdf to Sohaila Mohamed.pdf


In [44]:
import os

data_dir = "/content/data"
os.makedirs(data_dir, exist_ok=True)

# Move uploaded files
for file_name in uploaded.keys():
    os.rename(file_name, os.path.join(data_dir, file_name))


In [45]:
import os

os.environ["GROQ_API_KEY"] = "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"
os.environ["STORAGE_PATH"] = "/content/storage"
os.environ["DATA_PATH"] = "/content/data"


In [49]:
from flask import Flask, request, jsonify
import threading

app = Flask(__name__)

@app.route("/query", methods=["POST"])
def query():
    content = request.get_json(silent=True)
    q = content["query"]
    try:
        res = query_llm(q)
        return jsonify({"response": str(res)})
    except Exception as err:
        return jsonify({"error": str(err)}), 500

# Run Flask in a separate thread
def run_flask():
    app.run(host="0.0.0.0", port=5000)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

print("Flask is running in the background! You can run other cells now.")

Flask is running in the background! You can run other cells now.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [73]:
import requests
import json

url = "http://127.0.0.1:5000/query"
payload = {"query": "what is this pdf about ?"}

response = requests.post(url, json=payload)
response_data = response.json()  # Convert to dictionary

print(json.dumps(response_data, indent=4))


INFO:werkzeug:127.0.0.1 - - [27/Feb/2025 21:00:00] "POST /query HTTP/1.1" 200 -


The PDF is about the professional experience, skills, and projects of an individual named Sohaila Mohamed. It includes her roles in public relations, organization and logistics, media and marketing, and software engineering, as well as her self-learning initiatives in deep learning and Python programming. The document also highlights her core skills in deep learning, data augmentation, image registration, and more, along with her projects such as a defibrillator device, an obstacle avoidance wheelchair, and a gel electrophoresis system. Additionally, it mentions her volunteering work as a software engineer trainee.
{
    "response": "The PDF is about the professional experience, skills, and projects of an individual named Sohaila Mohamed. It includes her roles in public relations, organization and logistics, media and marketing, and software engineering, as well as her self-learning initiatives in deep learning and Python programming. The document also highlights her core skills in dee